In [ ]:
import random 
import numpy as np
import math 
from sympy import *
import time

In [ ]:
a1 = -15
a2 = 16
cat_1 = ["+", "-", "/", "*"] #["+", "-", "/", "*"]
cat_2 = ["exp"] #["sin", "cos", "exp", "ln"]
cat_3 = ["x", "c1", "c2", "c3"] #["x", "c1", "c2", "c3"]
cat_4 = ["(", ")"]
c1, c2, c3, x = symbols('c1 c2 c3 x')
dy2, dy, y, x = symbols('dy2 dy y x')

In [ ]:
def Tree(length):
  vec = []
  k = 0
  cat = np.random.randint(1,3)
  if (cat == 1): #проверка на функции
    vec += [cat_2[np.random.randint(0,len(cat_2))]]
  if (cat == 2): #проверка на терминалы
    vec += [random.choices(cat_3, weights=(70, 10, 10, 10))[0]]
  if (cat == 3): #проверка на скобки
    vec += [cat_4[0]]
    k += 1
  for i in range(1, length):
    if (i+k<length-1):
      cat = np.random.randint(1,2)
      if vec[i-1] in cat_1: #проверка на операции
        if (cat == 1): 
          vec += [cat_2[np.random.randint(0,len(cat_2))]]
        if (cat == 2): 
          vec += [random.choices(cat_3, weights=(70, 10, 10, 10))[0]]
      if vec[i-1] in cat_2: #проверка на функции
        vec += [cat_4[0]]
        k += 1
      if vec[i-1] in cat_3: #проверка на терминал
        if (random.choices((0,1)) == 0):
          while (k>0):
            vec += [cat_4[1]]
            k -= 1
          break
        else:
          vec += [cat_1[np.random.randint(0,len(cat_1))]]
      if vec[i-1] in cat_4[0]:
        vec += [random.choices(cat_3, weights=(70, 10, 10, 10))[0]]
      if vec[i-1] in cat_4[1]:
        vec += [cat_1[random.randint(0,len(cat_1))]]
    else:
      if (vec[i-1] in cat_1):
        vec += [random.choices(cat_3, weights=(70, 10, 10, 10))[0]]
      if (vec[i-2] in cat_2):
        #print('Бракованный индивид')
        vec.clear()
        vec = ['x']
        break
      while (k>0):
        vec += [cat_4[1]]
        k -= 1
        break
  vec0 = "".join(vec)
  return vec0, vec

In [ ]:
#размер популяции, размер "дерева"
def Generation(k, h):
  individ = []
  individ_str = []
  for i in range(k):
    tree = Tree(h)
    individ += [tree[0]]
    individ_str += [tree[1]]
    while (len(individ[i]) < 2):
      tree = Tree(h)
      individ.pop(i)
      individ_str.pop(i)
      individ += [tree[0]]
      individ_str += [tree[1]]  
  return individ, individ_str

In [ ]:
#указывается популяция, левая часть равенства, начальное условие, начало отрезка, конец отрезка, количество узлов
def Fit(population, popul_str, F0, a, b, N, k, h, y0, y1=None):
  t_fit= time.perf_counter()
  while(time.perf_counter() - t_fit < 60):
    l = len(population)
    #print(type(popul_str[0][0]))
    obj_func = np.zeros(l)
    E = np.zeros(l)
    fit = np.zeros(l)
    for i in range(l):
      F = []
      f = []
      df = []
      F1 = sympify(F0)
      y0 = sympify(y0)
      k2 = 1 #штраф за начальные условия
      k1 = 1 #штраф за сложность дерева
      D = len(popul_str[i]) # кол-во вершин дерева
      f = sympify(population[i])
      #print(population[i])
      c11 = round(np.random.uniform(a1, a2),2)
      c12 = round(np.random.uniform(a1, a2),2)
      c13 = round(np.random.uniform(a1, a2),2)
      for j in range(len(popul_str[i])):
        popul_str[i][j] = popul_str[i][j].replace("c1", str(c11)).replace("c2", str(c12)).replace("c3", str(c13))
      f = f.subs([(c1, c11),(c2, c12), (c3, c13)])
      population[i] = f
      if (y1 != None):
        df = f.diff(x)
        df2 = df.diff(x)
        F1 = F1.subs([(dy2,df2),(dy,df),(y,f)])
        for j in range(N+1):
          F += [re(F1.evalf(subs={x: a+j*(b-a)/N}))]
        F = np.array(F)
        Y0 = float(y0.evalf(subs={x:a}))
        Y1 = float(y1.evalf(subs={x:a}))
      else:
        df = f.diff(x)
        F1 = F1.subs([(dy,df),(y,f)])
        for j in range(N+1):
          F += [re(F1.evalf(subs={x: a+j*(b-a)/N}))]
        F = np.array(F)
        Y0 = float(y0.evalf(subs={x:a}))
      while (math.isnan(np.sum(F)) == True or abs(np.sum(F)) > 10 ** 3):
        F1 = sympify(F0)
        F = []
        f = []
        df = []
        population[i], popul_str[i] = Generation(k, h)
        population[i] = population[i][0]
        popul_str[i] = popul_str[i][0]
        f = sympify(population[i])
        c11 = round(np.random.uniform(a1, a2),2)
        c12 = round(np.random.uniform(a1, a2),2)
        c13 = round(np.random.uniform(a1, a2),2)
        for j in range(len(popul_str[i])):
          popul_str[i][j] = popul_str[i][j].replace("c1", str(c11)).replace("c2", str(c12)).replace("c3", str(c13))
        f = f.subs([(c1, c11),(c2, c12), (c3, c13)])
        population[i] = f
        if (y1 != None):
          df = f.diff(x)
          df2 = df.diff(x)
          F1 = F1.subs([(dy2,df2),(dy,df),(y,f)])
          for j in range(N+1):
            F += [re(F1.evalf(subs={x: a+j*(b-a)/N}))]
          F = np.array(F)
          Y0 = float(y0.evalf(subs={x:a}))
          Y1 = float(y1.evalf(subs={x:a}))
          Y1_1 = float(re(df.evalf(subs={x:a})))
        else:
          df = f.diff(x)
          F1 = F1.subs([(dy,df),(y,f)])
          for j in range(N+1):
            F += [re(F1.evalf(subs={x: a+j*(b-a)/N}))]
          F = np.array(F)
      #print("E=", k2*(Y0-F[0]) + abs(np.sum(F))/N)
      if (y1 != None):
        E[i] = k2*(Y0-F[0]+(Y1-Y1_1)) + abs(np.sum(F))/N
      else:
        E[i] = k2*(Y0-F[0]) + abs(np.sum(F))/N
      fit[i] = E[i] #1/(1+E[i]+k1*D)
    return fit
  else:
    return 0

In [ ]:
def Object_Function(individ, F0, a, b, N, y0, y1=None):
  t_of = time.perf_counter()
  while(time.perf_counter() - t_of < 120): 
    F = []
    df = individ.diff(x)
    if (y1 != None):
        df2 = df.diff(x)
        F1 = F1.subs([(dy2,df2),(dy,df),(y,individ)])
        for j in range(N+1):
          F += [re(F1.evalf(subs={x: a+j*(b-a)/N}))]
        F = np.array(F)
        Y0 = float(y0.evalf(subs={x:a}))
        Y1 = float(y1.evalf(subs={x:a}))
        Y1_1 = float(re(df.evalf(subs={x:a})))
        return np.sum(F[:N]), (Y0 - float(F[0]))*100 + (Y1 - Y1_1)*100
    else:
      F1 = sympify(F0)
      F1 = F1.subs([(dy,df),(y,individ)])
      for j in range(N+1):
          F += [re(F1.evalf(subs={x: a+j*(b-a)/N}))]
      y0 = sympify(y0)
      Y0 = float(y0.evalf(subs={x:a}))
      F = np.array(F)
      return np.sum(F[:N]), (Y0 - float(F[0]))*100
  else:
    return nan, nan

In [ ]:
#передается популяция, начало и конец отрезка
def Optimizate(population, popul_str, F0, a, b, N):
  t_opt = time.perf_counter()
  mod_pop = population
  mod_pop_str = popul_str
  while(time.perf_counter() - t_opt < 100):
    new_popul_str = []
    for i in range(len(population)):
      new_popul_str = []
      l = np.random.randint(0,len(popul_str[i]))
      if (popul_str[i][l] in cat_1):
        for j in range(len(cat_1)):
          new_popul_str = popul_str[i][:l]
          new_popul_str  += cat_1[j]
          new_popul_str += popul_str[i][l+1:]
          individ = sympify("".join(new_popul_str))
          for d in preorder_traversal(individ):
            if isinstance(d, Float):
              individ = individ.subs(d, round(d, 2))
          if (abs(Object_Function(individ, F0, a, b, N, y0, y1)[0]) < abs(Object_Function(population[i],  F0, a, b, N, y0, y1)[0]) and abs(Object_Function(individ,  F0, a, b, N, y0, y1)[1]) < abs(Object_Function(population[i],  F0, a, b, N, y0, y1)[1])):
            population[i] = individ
            popul_str[i] = new_popul_str
            break
          else:
            individ = []
            new_popul_str = []
      if (popul_str[i][l] in cat_2):
        for j in range(len(cat_2)):
          new_popul_str = popul_str[i][:l]
          new_popul_str  += cat_2[j]
          new_popul_str += popul_str[i][l+1:]
          individ = sympify("".join(new_popul_str))
          for d in preorder_traversal(individ):
            if isinstance(d, Float):
              individ = individ.subs(d, round(d, 2))
          if (abs(Object_Function(individ, F0, a, b, N, y0, y1)[0]) < abs(Object_Function(population[i], F0, a, b, N, y0, y1)[0]) and abs(Object_Function(individ,  F0, a, b, N, y0, y1)[1]) < abs(Object_Function(population[i],  F0, a, b, N, y0, y1)[1])):
            population[i] = individ
            popul_str[i] = new_popul_str
            break
          else:
            individ = []
            new_popul_str = []
      if (popul_str[i][l] in cat_3):
        new_popul_str = popul_str[i][:l+1]
        new_popul_str += '+'
        new_popul_str += [str(round(np.random.uniform(a, b),2))]
        new_popul_str += popul_str[i][l+1:]
        individ = sympify("".join(new_popul_str))
        for d in preorder_traversal(individ):
            if isinstance(d, Float):
              individ = individ.subs(d, round(d, 2))
        if (abs(Object_Function(individ, F0, a, b, N, y0, y1)[0]) < abs(Object_Function(population[i], F0, a, b, N, y0, y1)[0]) and abs(Object_Function(individ,  F0, a, b, N, y0, y1)[1]) < abs(Object_Function(population[i],  F0, a, b, N, y0, y1)[1])):
          population[i] = individ
          popul_str[i] = new_popul_str
        else:
          individ = []
          new_popul_str = []
          new_popul_str = popul_str[i][:l+1]
          new_popul_str  += '-'
          new_popul_str += [str(round(np.random.uniform(a, b),2))]
          new_popul_str += popul_str[i][l+1:]
          individ = sympify("".join(new_popul_str))
          for d in preorder_traversal(individ):
            if isinstance(d, Float):
              individ = individ.subs(d, round(d, 2))
          if (abs(Object_Function(individ, F0, a, b, N, y0, y1)[0]) < abs(Object_Function(population[i], F0, a, b, N, y0, y1)[0]) and abs(Object_Function(individ,  F0, a, b, N, y0, y1)[1]) < abs(Object_Function(population[i],  F0, a, b, N, y0, y1)[1])):
            population[i] = individ
            popul_str[i] = new_popul_str
          else:
            individ = []
            new_popul_str = []
        new_popul_str = popul_str[i][:l+1]
        new_popul_str += '*'
        new_popul_str += [str(random.choice([-4, -3, -2, -1, 1, 2, 3 , 4]))]
        new_popul_str += popul_str[i][l+1:]
        individ = sympify("".join(new_popul_str))
        for d in preorder_traversal(individ):
            if isinstance(d, Float):
              individ = individ.subs(d, round(d, 2))
        if (abs(Object_Function(individ, F0, a, b, N, y0, y1)[0]) < abs(Object_Function(population[i], F0, a, b, N, y0, y1)[0]) and abs(Object_Function(individ,  F0, a, b, N, y0, y1)[1]) < abs(Object_Function(population[i],  F0, a, b, N, y0, y1)[1])):
          population[i] = individ
          popul_str[i] = new_popul_str
        else:
          individ = []
          new_popul_str = []
          new_popul_str = popul_str[i][:l+1]
          new_popul_str  += '-'
          new_popul_str += [str(round(np.random.uniform(a, b),2))]
          new_popul_str += popul_str[i][l+1:]
          individ = sympify("".join(new_popul_str))
          for d in preorder_traversal(individ):
            if isinstance(d, Float):
              individ = individ.subs(d, round(d, 2))
          if (abs(Object_Function(individ, F0, a, b, N, y0, y1)[0]) < abs(Object_Function(population[i], F0, a, b, N, y0, y1)[0]) and abs(Object_Function(individ,  F0, a, b, N, y0, y1)[1]) < abs(Object_Function(population[i],  F0, a, b, N, y0, y1)[1])):
            population[i] = individ
            popul_str[i] = new_popul_str
          else:
            individ = []
            new_popul_str = []
    return population, popul_str
  else:
    print('ÖRTHT = ', time.perf_counter() - t_start)
    return mod_pop, mod_pop_str

In [ ]:
def Crossing(popul_str_1, popul_str_2, population_1, population_2, p):
  t_cross = time.perf_counter()
  mod_pop_1 = population_1
  mod_pop_2 = population_2
  mod_pop_str_1 = popul_str_1
  mod_pop_str_2 = popul_str_2
  while(time.perf_counter() - t_cross < 120):
    if (np.random.uniform(0, 1) > p):
      if (abs(Object_Function(population_1, F0, a, b, N, y0, y1)[0]) >= abs(Object_Function(population_2, F0, a, b, N, y0, y1)[0])):
        return population_2, popul_str_2
      else:
        return population_1, popul_str_1
    else:
      f = False
      k1 = 0
      k2 = 0
      l1 = []
      l2 = []
      for i in range(3, len(popul_str_1)-2): 
        if (popul_str_1[i] in cat_1):
          l1 += [i]
      for i in range(3, len(popul_str_2)-2):
        if (popul_str_2[i] in cat_1):
          l2 += [i]
      for i in l1:
        for j in l2:
          if (popul_str_1[i] == popul_str_2[j]): #проверка на содержание совпадающих элементов из первой категории
            f = True
            break
        if (f == True):
          break
      if (f == True):
        z1 = l1[np.random.randint(0, len(l1))]
        z2 = l2[np.random.randint(0, len(l2))]
        r = 0
        while (r < 1): 
          if (popul_str_1[z1] == popul_str_2[z2]): # совпадают ли элементы, в которых будет совершаться разрыв
            r += 1
          else:
            r = 0
            z1 = l1[np.random.randint(0, len(l1))]
            z2 = l2[np.random.randint(0, len(l2))]
      else:
        z1 = l1[np.random.randint(0, len(l1))] 
        z2 = l2[np.random.randint(0, len(l2))]
      #new_popul_str = popul_str_1[:z1] + popul_str_2[z2:]
      l1 = []
      l2 = []
      for i in range(z1):
        if (popul_str_1[i] in cat_4[0]):
          l1 += [i]
          k1 += 1
        if (popul_str_1[i] in cat_4[1]):
          l2 += [i]
          k1 -= 1
      while (k1 > 0):
        if (len(l2)>0):
          new_popul_str0 = []
          new_popul_str0 += popul_str_1[:l2[np.random.randint(0, len(l2))]+1]
          new_popul_str0 += ')'
          new_popul_str0 += popul_str_1[l2[np.random.randint(0, len(l2))]+1:z1]
          k1 -= 1
          new_popul_str = new_popul_str0
        else:
          new_popul_str = pop
      while (k1 < 0):
        new_popul_str0 = []
        new_popul_str0 += popul_str_1[:l1[np.random.randint(0, len(l1))]+1]
        new_popul_str0 += '('
        new_popul_str0 += popul_str_1[l1[np.random.randint(0, len(l1))]+1:z1]
        k1 += 1
        new_popul_str = new_popul_str0
      k = 0
      l1 = []
      l2 = []
      new_popul_str = popul_str_1[:z1]
      for i in range(z2,len(popul_str_2)):
        if (popul_str_2[i] in cat_4[0]):
          l1 += [i]
          k += 1
        if (popul_str_2[i] in cat_4[1]):
          l2 += [i]
          k -= 1
      while (k > 0):
        new_popul_str0 = []
        new_popul_str0 += popul_str_2[z2:l2[np.random.randint(0, len(l2))]+1]
        new_popul_str0 += ')'
        new_popul_str0 += popul_str_2[l2[np.random.randint(0, len(l2))]+1:]
        k -= 1
        new_popul_str = new_popul_str0
      while (k < 0):
        new_popul_str0 = []
        new_popul_str0 += popul_str_1[z2:l1[np.random.randint(0, len(l1))]+1]
        new_popul_str0 += '('
        new_popul_str0 += popul_str_1[l1[np.random.randint(0, len(l1))]+1]
        k += 1
        new_popul_str = new_popul_str0
      new_popul = sympify("".join(new_popul_str))
      #print(new_popul, new_popul_str)
      return new_popul, new_popul_str
  else:
    if (abs(Object_Function(mod_pop_1, F0, a, b, N, y0, y1)[0]) >= abs(Object_Function(mod_pop_2, F0, a, b, N, y0, y1)[0])):
      return mod_pop_2, mod_pop_str_2
    else:
      return mod_pop_1, mod_pop_str_1

In [ ]:
for i in range(len(new_popul_str)):
        if (new_popul_str[i] in cat_4[0]):
          k += 1
        if (new_popul_str[i] in cat_4[1]):
          k -= 1
      while (k > 0):
        new_popul_str += [')']
        k -= 1
      while (k < 0):
        for j in range(len(new_popul_str)):
          if (new_popul_str[j] in cat_4[0]):
            l3 += [j]
        print('NEW_POPUL = ',k, '   ', new_popul_str )
        new_popul_str0 = []
        new_popul_str0 += new_popul_str[:l3[np.random.randint(0, len(l3))]+1]
        new_popul_str0 += '('
        new_popul_str0 += new_popul_str[l3[np.random.randint(0, len(l3))]+1:]
        k += 1
        new_popul_str = new_popul_str0
while (k < 0):
        for j in range(len(new_popul_str)):
          if (new_popul_str[j] in cat_4[0]):
            l3 += [j]
        print('NEW_POPUL = ',k, '   ', new_popul_str )
        new_popul_str0 = []
        new_popul_str0 += new_popul_str[:l3[np.random.randint(0, len(l3))]+1]
        new_popul_str0 += '('
        new_popul_str0 += new_popul_str[l3[np.random.randint(0, len(l3))]+1:]
        k += 1
        new_popul_str = new_popul_str0

In [ ]:
# популяция и вероятность скрещивания
def Genetic_Algorithm(population, popul_str, p):
  # selection турнирная
  new_population = [] # популяция новых индивидов в sympy
  new_popul_str = [] # популяция новых индивидов в строковом виде
  while (len(new_population) < len(population)): # сравниваю заполненность новой популяции
    cross_str = []
    cross = []
    while (len(cross) < 2): # массив для скрещивания
      r1 = np.random.randint(0,len(population))
      r2 = np.random.randint(0,len(population))
      while (r1 == r2): # добиваюсь неравенства индивидов
        r1 = np.random.randint(0,len(population))
        r2 = np.random.randint(0,len(population))
      if (abs(Object_Function(population[r1], F0, a, b, N, y0, y1)[0]) >= abs(Object_Function(population[r2], F0, a, b, N, y0, y1)[0]) and abs(Object_Function(population[r1], F0, a, b, N, y0, y1)[1]) >= abs(Object_Function(population[r2], F0, a, b, N, y0, y1)[1])): # турнирный отбор: наиболее приспособленные индвиды идут дальше
        cross_str.append(popul_str[r2])
        cross.append(population[r2])
      else:
        cross_str.append(popul_str[r1])
        cross.append(population[r1])
  # Cross
    new_popul = [Crossing(cross_str[0], cross_str[1], cross[0], cross[1], p)][0] #скрещивание
    new_population += [new_popul[0]]
    new_popul_str += [new_popul[1]] #получаем новую популяцию
  # конец while
  # mutation
  population = new_population
  popul_str = new_popul_str
  for i in range(len(population)):
    new_popul_str = []
    k = 0
    r = []
    for j in range(len(popul_str[i])):
      if (popul_str[i][j] == 'x'):
        r += [j]
    if (len(r) == 0):
      continue
    else:
      l = r[np.random.randint(len(r))]
      if (np.random.uniform(a, b) > 0.5):
        new_popul_str = popul_str[i][:l]
        new_popul_str += '('
        new_popul_str += popul_str[i][l]
        new_popul_str += '+'
        new_popul_str += [str(random.choice([-4, -3, -2, -1, 1, 2, 3 , 4]))]
        new_popul_str += ')'
        new_popul_str += popul_str[i][l+1:]
        individ = sympify("".join(new_popul_str))
        population[i] = individ
        popul_str[i] = new_popul_str
        new_popul_str = [] 
        individ = []
  return population, popul_str

In [ ]:
F0 = '(x+1)*dy+x*y' # дифференциальное уравнение
y0 ='1.0' #начальное условие
y1 = None
a = 0 #начало отрезка
b = 3 #конец отрезка
N = 50 #количество разбиений в сетке
k = 10 #размер популяции
h = 10 #размерность "дерева"

In [ ]:
# получение популяции и приспособленность особей
while True:
  try:
    population, popul_str = Generation(k, h)
    fit = 0
    while (type(fit) == int):
      fit = Fit(population, popul_str, F0, a, b, N, k, h, y0, y1)
  except TypeError as e:
    if str(e) == "cannot unpack non-iterable NoneType object":
      continue
    else:
      raise e
  except ZeroDivisionError:
    continue
  else:
    print(population)
    print(fit)
    break

[exp(x - exp(x)), exp(2.02488023978819e-6*x), exp(x - exp(x)), exp(x - exp(x)), exp(x - exp(x)), 0.989333240569584, 1.00340267903861, exp(-12.75*exp(-x)), exp(x - exp(x)), exp(-4.53*exp(-x))]
[1.17212497 2.5300094  1.17212497 1.17212497 1.17212497 2.51367986
 2.5352061  1.93843207 1.17212497 2.45482472]


In [ ]:
while True:
  try:
    population, popul_str = Optimizate(population, popul_str, F0, a, b, N)
    fit = []
    for i in range(len(population)):
      fit += [Object_Function(population[i], F0, a, b, N, y0, y1)]
  except TypeError as e:
    if str(e) == "cannot unpack non-iterable NoneType object":
      continue
    else:
      raise e
  except ZeroDivisionError:
    continue
  else:
    print(population)
    print(fit)
    break

[exp(x - exp(x)), exp(2.02488023978819e-6*x), exp(x - exp(x)), exp(x - exp(x)), exp(x - exp(x)), 0.989333240569584, 1.00340267903861, exp(-12.75*exp(-x)), exp(x - exp(x)), exp(-4.53*exp(-x))]
[(-8.60624546813322, 100.0), (73.5005447550935, 99.99979751197601), (-8.60624546813322, 100.0), (-8.60624546813322, 100.0), (-8.60624546813322, 100.0), (72.7159931818645, 100.0), (73.7500969093380, 100.0), (43.9874355423252, 99.99629954147898), (-8.60624546813322, 100.0), (72.0688036486107, 95.11635373904738)]


In [ ]:
while True:
  try:
    flag = 0
    for i in range(5):
      print('       i=    ', i)
      t_start = time.perf_counter()
      population, popul_str = Optimizate(population, popul_str, F0, a, b, N)
      print('ПОСЛЕ ОПТИМИЗАЦИИ:')
      print(population, popul_str)
      fit = []
      for j in range(len(population)):
        fit += [Object_Function(population[j], F0, a, b, N, y0, y1)]
        if (abs(fit[j][0]) < 0.05 and abs(fit[j][1]) < 1):
          print('НАЙДЕННОЕ РЕШЕНИЕ= ', population[j])
          print('ОШИБКА= ', fit[j])
          flag = 1
          break
      if (flag == 1):
        break
      else: 
        population, popul_str = Genetic_Algorithm(population, popul_str, 0.8)
        print('генетический алгоритм:')
        print(population, popul_str)
        fit = []
        for j in range(len(population)):
          fit += [Object_Function(population[j], F0, a, b, N, y0, y1)]
          if (abs(fit[j][0]) < 0.05 and abs(fit[j][1]) < 1):
            print('НАЙДЕННОЕ РЕШЕНИЕ= ', population[j])
            print('ОШИБКА= ', fit[j])
            flag = 1
            break
        if (flag == 1):
          break
        t_stop = time.perf_counter()
        print('Время одной итерации в секундах = ', t_stop - t_start)
    else:
      break
  except TypeError as e:
    if str(e) == "cannot unpack non-iterable NoneType object":
      continue
    else:
      raise e
  except ZeroDivisionError:
    continue
  else:
    print(population)
    print(fit)
    break

       i=     0
ПОСЛЕ ОПТИМИЗАЦИИ:
[exp(x - exp(x - 4) - 3), 0.124930212198582*exp(x - exp(x)), exp(x - exp(x + 1) - 4), exp(x - exp(x - exp(x + 2) - 2) - 2), exp(x - 1.86*exp(x + 2) - 1), 2.5092903899363*exp(x - exp(x)), 0.0829099665751727*exp(x - exp(x - 4)), exp(x - exp(x - 2) - 1), exp(x - exp(x - 1) - 3), exp(x - exp(x) + 7)] [['exp', '(', '(', '(', 'x', '+', '-2', ')', '+', '-1', ')', '-', 'exp', '(', '(', '(', '(', 'x', '+', '-3', ')', '+', '1', ')', '+', '-2', ')', ')', ')'], ['exp', '(', '(', '(', '(', '(', 'x', '+', '2', ')', '-', '0.28', '-', '2.8', '+', '1', ')', '+', '-2', ')', '-', 'exp', '(', '(', '(', 'x', '+', '1', ')', '+', '-1', ')', ')', ')', ')'], ['exp', '(', '(', 'x', '+', '-4', '-', 'exp', '(', '(', '(', '(', '(', 'x', '+', '-2', ')', '+', '-2', ')', '+', '3', ')', '+', '2', ')', ')', ')', ')'], ['exp', '(', '(', '(', 'x', '+', '-2', ')', '-', 'exp', '(', '(', 'x', '+', '-2', ')', '-', 'exp', '(', '(', '(', 'x', '+', '3', ')', '+', '-1', ')', ')', ')', ')', ')']

ValueError: ignored